# Recommender systems on the Movielens dataset

A recommender system is a machine learning algorithm that seeks to predict the rating a user would give to an item. One approach to the design of recommender systems that has been widely used is *collaborative filtering*.

This problem has become quite popular a few years ago with the *Netflix Prize*: an open competition for the best collaborative filtering algorithm to predict user ratings for films, based on previous ratings (and without the users or the films being identified except by numbers assigned for the contest -- this is different from so-called content-based methods).

First, let us import the usual suspects

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py

import keras
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, dot, add
from keras.regularizers import l2
from keras.optimizers import adam

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


## Taking a look at the data

Instead of working with the dataset from the Netflix prize, we shall use instead the smaller one from Movielens. Let us load the  Movielens data and take a look at it using Pandas.

In [2]:
ratings = pd.read_csv("mldata/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


What we see is that user $1$ rated movie $31$ as 2.5 (quite average), but on the other hand likes movie $1172$ quite a lot.

In [3]:
len(ratings)

100004

The number of ratings is 100004. Just for fun, let us read the movie names as well. 

In [4]:
movie_names = pd.read_csv("mldata/movies.csv").set_index('movieId')['title']
movie_names

movieId
1                                          Toy Story (1995)
2                                            Jumanji (1995)
3                                   Grumpier Old Men (1995)
4                                  Waiting to Exhale (1995)
5                        Father of the Bride Part II (1995)
6                                               Heat (1995)
7                                            Sabrina (1995)
8                                       Tom and Huck (1995)
9                                       Sudden Death (1995)
10                                         GoldenEye (1995)
11                           American President, The (1995)
12                       Dracula: Dead and Loving It (1995)
13                                             Balto (1995)
14                                             Nixon (1995)
15                                  Cutthroat Island (1995)
16                                            Casino (1995)
17                             S

We now preprocess our data. We will update the user and movie indices so that they are contiguous integers -- this will simplify our analysis.

In [5]:
# Extract unique user and movie ids
users = ratings.userId.unique()
movies = ratings.movieId.unique()

# Create dictionaries mapping index to integer between 1 and n_users/n_movies
user_id2idx = {o:i for i, o in enumerate(users)}
movie_id2idx = {o:i for i, o in enumerate(movies)}

# Update indices 
ratings.movieId = ratings.movieId.apply(lambda x: movie_id2idx[x])
ratings.userId = ratings.userId.apply(lambda x: user_id2idx[x])

# Let us check what is the minimum and maximum indices now
print("min/max user indices: %g, %g" % (ratings.userId.min(), ratings.userId.max()))
print("min/max movie indices: %g, %g" % (ratings.movieId.min(), ratings.movieId.max()))

min/max user indices: 0, 670
min/max movie indices: 0, 9065


In [6]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
print("number of users/movies: %g, %g" % (n_users, n_movies)) # (this should give the same as above)

number of users/movies: 671, 9066


Finally, as we usually do, let us split our data into training and validation.

In [7]:
np.random.seed(42)
mask = np.random.rand(len(ratings)) < 0.8
train, val = ratings[mask], ratings[~mask]

## Let us look at a subset of the data

Just to get some intuition on our data, let's look at a subset of the matrix with the 15 users/movies who have the most votes associated to them.

In [8]:
# Create series containing vote count per user, then sort it and pick the first 15
users_votecount = ratings.groupby('userId')['rating'].count()
top_users = users_votecount.sort_values(ascending=False)[:15]
top_users.rename("votes", inplace=True)
top_users

userId
546    2391
563    1868
623    1735
14     1700
72     1610
451    1340
467    1291
379    1063
310    1019
29     1011
293     947
508     923
579     922
212     910
211     876
Name: votes, dtype: int64

In [9]:
# Create series containing vote count per movie, then sort it and pick the first 15
movies_votecount = ratings.groupby('movieId')['rating'].count()
top_movies = movies_votecount.sort_values(ascending=False)[:15]
top_movies.rename("votes", inplace=True)
top_movies

movieId
57     341
49     324
99     311
92     304
143    291
72     274
402    259
417    247
79     244
89     237
179    234
27     228
197    226
505    224
180    220
Name: votes, dtype: int64

In [10]:
# Create data frame with only top movies/users, and then a cross-tab
top_ratings = ratings.loc[ratings["movieId"].isin(top_movies.index) & ratings["userId"].isin(top_users.index)]
pd.crosstab(top_ratings["userId"], top_ratings["movieId"], top_ratings["rating"], aggfunc=sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Matrix factorization approach

One of the simplest models for recommender systems, which has been (and still is) a very popular one, is that of low-rank matrix factorization.

In this model, one assumes the presence of *latent factors* for each movie and user, that will generate the final rating.
More precisely, we look for a factorization of the true rating matrix, i.e. we are trying to find two matrices $U$ and $V$ such that the true rating matrix $Y$ is represented as

$$Y=UV^T + Z
\qquad\text{or, in components,}\qquad
Y_{ia} = \vec U_i \cdot \vec V_a + Z_{ia} = \sum_p U_{ip} V_{ap} + Z_{ia}$$

Here we label users by $i$ and movies by $a$.
The matrix $Z$ will be interpreted as a Gaussian noise term:
then, training amounts to minimize the mean square error

$$MSE = \sum_{(ia)\in {\cal T}} (Y_{ia} - (U V^T)_{ia} )^2$$

using the pairs $(ia)\in {\cal T}$ in the training set. Once the matrices have been trained, 
we can use $Y_{ia} = \sum_p U_{ip} V_{ap}$ to predict the unknown rating of movie $a$ by user $i$.



### Network architecture and Keras implementation

Somewhat surprisingly, Keras/Tensorflow can also solve problems like this! Let's see how well that works. 
We are not going to use a `Sequential()` model anymore, but instead the more general `Model()`. We will employ four different types of layers: `Input` and `Embedding` layers for the users and for the movies, and `Dot` and `Flatten` layers combining them.

Assuming that $U$ and $V$ have rank $50$ (so that we have $50$ latent variables for each users/movies), we have

In [11]:
# Parameters
n_factors = 50

# Specify model in Keras using embedding layers
user_input = Input(shape=(1,), dtype='int64', name='user')
U = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_input)
movie_input = Input(shape=(1,), dtype='int64', name='movie')
V = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_input)
Y = dot([U, V], axes=-1)
Y_r = Flatten()(Y)

model = Model([user_input, movie_input], Y_r)
model.compile(adam(lr=1e-3), loss='mse')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 50)        33550       user[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 50)        453300      movie[0][0]                      
__________________________________________________________________________________________________
dot_1 (Dot

In [12]:
epochs=5
batch_size = 64

# Train model
model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, epochs=epochs, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80099 samples, validate on 19905 samples
Epoch 1/5
80099/80099 [==============================] - 9s 111us/step - loss: 10.1458 - val_loss: 4.4030
Epoch 2/5
80099/80099 [==============================] - 9s 110us/step - loss: 3.1957 - val_loss: 2.8301
Epoch 3/5
80099/80099 [==============================] - 9s 108us/step - loss: 2.4148 - val_loss: 2.6164
Epoch 4/5
80099/80099 [==============================] - 9s 109us/step - loss: 2.2318 - val_loss: 2.5680
Epoch 5/5
80099/80099 [==============================] - 9s 110us/step - loss: 2.1599 - val_loss: 2.5541


Let's reduce the learning rate before we start overfitting

In [13]:
model.optimizer.lr = 1e-4
epochs=3

model.fit([train.userId, train.movieId], train.rating, batch_size=batch_size, epochs=epochs, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80099 samples, validate on 19905 samples
Epoch 1/3
80099/80099 [==============================] - 8s 102us/step - loss: 2.1197 - val_loss: 2.5612
Epoch 2/3
80099/80099 [==============================] - 8s 102us/step - loss: 2.0900 - val_loss: 2.5622
Epoch 3/3
80099/80099 [==============================] - 9s 107us/step - loss: 2.0669 - val_loss: 2.5679


We are still minimizing the training loss, but at this point we are clearly ovefitting since the validation loss is increasing... So our model does not seems so good.  On similar data sets, the [best benchmarks](http://www.librec.net/example.html) are a bit above 0.9, so something is not right here.

##  Adding a bias to users and movies

Clearly, we have a problem. We have forgotten that not all users are equal, and not all movies are equal. Some users always rate movies high, other always rates them low: each person has a different reference frame. The same is true for movies: everyone likes The Godfather, so we do not take much risk when we predict someone wants to watch it.

This can be fixed by introducing **bias terms** - that is, a different *bias* for each user and each movie, representing how positive or negative user votes typically are, and how good or bad each movie is typically considered to be. 

To take into account the bias we modify the model as

$$Y_{ia} = B_i + B_a + \vec U_i \cdot \vec V_a + Z_{ia}$$

We can add that easily by simply creating an additional embedding with one output for each movie and each user, and adding it to our output.

In [14]:
# Define model as before
user_input = Input(shape=(1,), dtype='int64', name='user')
U = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_input)
movie_input = Input(shape=(1,), dtype='int64', name='movie')
V = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_input)
Y = dot([U, V], axes=-1)
Y_r = Flatten()(Y)

# Define bias terms and add them to output
user_bias = Flatten()(Embedding(n_users, 1, input_length=1)(user_input))
movie_bias = Flatten()(Embedding(n_movies, 1, input_length=1)(movie_input))
Y_b = add([Y_r, user_bias, movie_bias])

# Compile model
model = Model([user_input, movie_input], Y_b)
model.compile(adam(lr=1e-3), loss='mse')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 50)        33550       user[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 50)        453300      movie[0][0]                      
__________________________________________________________________________________________________
dot_2 (Dot

In [15]:
# Train model
model.fit([train.userId, train.movieId], train.rating, batch_size=64, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80099 samples, validate on 19905 samples
Epoch 1/10
80099/80099 [==============================] - 9s 109us/step - loss: 8.8080 - val_loss: 3.4899
Epoch 2/10
80099/80099 [==============================] - 9s 110us/step - loss: 2.5869 - val_loss: 2.3053
Epoch 3/10
80099/80099 [==============================] - 7s 93us/step - loss: 1.9951 - val_loss: 2.1133
Epoch 4/10
80099/80099 [==============================] - 8s 98us/step - loss: 1.8328 - val_loss: 2.0188
Epoch 5/10
80099/80099 [==============================] - 8s 96us/step - loss: 1.7372 - val_loss: 1.9406
Epoch 6/10
80099/80099 [==============================] - 8s 101us/step - loss: 1.6564 - val_loss: 1.8739
Epoch 7/10
80099/80099 [==============================] - 9s 109us/step - loss: 1.5799 - val_loss: 1.8087
Epoch 8/10
80099/80099 [==============================] - 9s 107us/step - loss: 1.5074 - val_loss: 1.7412
Epoch 9/10
80099/80099 [==============================] - 9s 107us/step - loss: 1.4369 - val_loss: 1.6795

Let us try using smaller and smaller learning rates until we see we are overfitting.

In [ ]:
model.optimizer.lr = 1e-4
model.fit([train.userId, train.movieId], train.rating, batch_size=64, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
model.optimizer.lr = 5e-5
model.fit([train.userId, train.movieId], train.rating, batch_size=64, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

In [ ]:
model.optimizer.lr = 1e-5
model.fit([train.userId, train.movieId], train.rating, batch_size=64, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Seems like we are overfitting now... Still this result is quite respectable: our validation loss is about 1, so our error is around one of the benchmarks we could find with a quick Google search. So this looks like a great approach!

In [ ]:
model.save_weights("mldata/bias.h5")
model.load_weights("mldata/bias.h5")

We can use the model to generate predictions by passing a pair of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [ ]:
model.predict([np.array([3]), np.array([6])])

but not movie 100...

In [ ]:
model.predict([np.array([3]), np.array([100])])

It actually makes sense, if you ask me!

In [ ]:
print(movie_names[movies[6]], " ", movie_names[movies[100]])

## Analyze and interpret the results

To make the analysis of the factors more interesting, we shall restrict it to the top 2000 most popular movies.

In [ ]:
movies_votecount = ratings.groupby('movieId')['rating'].count()
top_movies = movies_votecount.sort_values(ascending=False)[:2000]
top_movies.rename("votes", inplace=True)

# Let us create at data frame and handle all data together
top_movies = pd.DataFrame(top_movies)
top_movies["title"] = movie_names[movies[top_movies.index]].values
top_movies

First, we'll look at the movie bias term. We create a "model" - which in Keras is simply a way of associating one or more inputs with one or more outputs, using the functional API. Here, our input is the movie id (a single id), and the output is the movie bias (a single float).

We can then look at the top and bottom rated movies. These ratings are corrected for different levels of reviewer sentiment, as well as different types of movies that different reviewers watch.

In [ ]:
get_movie_bias = Model(movie_input, movie_bias)
top_movies["bias"] = get_movie_bias.predict(top_movies.index)
top_movies.sort_values("bias")

These are indeed quite bad if you ask me...

In [ ]:
top_movies.sort_values("bias", ascending=False)

and those are quite good ones! We can now do the same thing at the level of the embeddings to see what are the *features* that have been learned.

In [ ]:
get_movie_embed = Model(movie_input, V)
movie_embed = np.squeeze(get_movie_embed.predict([top_movies.index]))
movie_embed.shape

Because it's hard to interpret $50$ embeddings, we use [PCA](https://plot.ly/ipython-notebooks/principal-component-analysis/) to simplify them down to just $3$ vectors. 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_embed.T).components_

Here's the 1st component. It seems to be related to how "critically acclaimed" or "classic" the movie is...

In [ ]:
top_movies["pca_1"] = movie_pca[0]
top_movies.sort_values("pca_1")

The 2nd looks like "blockbuster".

In [ ]:
top_movies["pca_2"] = movie_pca[1]
top_movies.sort_values("pca_2")

The 3rd seems to be looking at violent vs. happy.

In [ ]:
top_movies["pca_3"] = movie_pca[2]
top_movies.sort_values("pca_3")

We can draw a picture to see how various movies appear on the map of these PCA components. This picture will show us the 1st and 3rd components.

In [ ]:
start = 0
end = 30

xx = movie_pca[0, start:end]
yy = movie_pca[2, start:end]
plt.figure(figsize=(15, 15))
plt.scatter(xx, yy)
for title, x, y in zip(top_movies.iloc[start:end]["title"].values, xx, yy):
    plt.text(x + 1e-3, y + 1e-3, title, color="k", fontsize=16)
plt.show()

##  Moving to neural nets

Now that we have tried to be clever, using the kind of techniques that were used for the Netflix prize, we should try to blindly use generic powerful techniques, like... a neural network! Indeed, rather than creating a *special purpose* architecture (like our matrix factorization with bias), it's often both easier and more accurate to use a standard neural network.

Let's try it! Here, we simply concatenate the user and movie embeddings into a single vector, which we feed into the neural net.

In [ ]:
from keras.layers import concatenate, Dense, Dropout

# Generate embeddings and concatenate them
user_input = Input(shape=(1,), dtype='int64', name='user')
U = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_input)
movie_input = Input(shape=(1,), dtype='int64', name='movie')
V = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_input)
Y = concatenate([U, V])
Y_r = Flatten()(Y)

# Specify neural network architecture
Y_nn = Dropout(0.3)(Y_r)
Y_nn = Dense(70, activation='relu')(Y_nn)
Y_nn = Dropout(0.75)(Y_nn)
Y_nn = Dense(1)(Y_nn)
nn = Model([user_input, movie_input], Y_nn)

nn.summary()

In [ ]:
# Train model
nn.compile(adam(0.001), loss='mse')
nn.fit([train.userId, train.movieId], train.rating, batch_size=64, epochs=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Boom! This improves on our accuracy even further right away! The power of neural nets is definitely impressive! At this point you can try to play with the architecture and see if you can improve on it again.

Note it is a bit harder to interpret these results: we wouldn't be able to do an analysis like the one we did in the last section. This is one the big issues in deep learning today: *interpretability*.